# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read in .csv to pandas dataframe                    
weather_df = pd.read_csv( '../output_data/cities.csv')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Folkestone,51.0833,1.1833,43.38,70,75,18.41,GB,1619072137
1,Punta Arenas,-53.1500,-70.9167,41.00,87,90,8.05,CL,1619071844
2,Buraidah,26.3260,43.9750,86.00,25,0,3.44,SA,1619072137
3,Lagoa,39.0500,-27.9833,59.81,91,57,31.03,PT,1619072138
4,Tabuk,17.4189,121.4443,86.67,58,100,10.87,PH,1619072138


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure( api_key = g_key)

In [4]:
# Store latitude and longitude
locations = weather_df[[ 'Lat', 'Lng']]

# Store Humidity
humidity = weather_df[ 'Humidity'].astype( 'float')

In [5]:
# Plot Heatmap
fig = gmaps.figure(center = (0,0), zoom_level= 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity, 
                                 dissipating = False, 
                                 max_intensity = 100, 
                                 point_radius = 3
                                )

# Add layer
fig.add_layer( heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 15 mph, cloudiness equals to 0 and max temp between 75 and 85
optimal_df = weather_df.loc[( weather_df[ 'Wind Speed'] <= 15) & ( weather_df[ 'Cloudiness'] == 0) & \
                                   ( weather_df[ 'Max Temp'] >= 75) & ( weather_df[ 'Max Temp'] <= 85)].dropna()

optimal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
232,Maceió,-9.6658,-35.7353,75.20,94,0,4.61,BR,1619072226
262,Yulara,-25.2406,130.9889,78.80,17,0,8.05,AU,1619072238
322,Nicoya,10.1483,-85.4520,78.80,65,0,9.22,CR,1619072262
370,Alice Springs,-23.7000,133.8833,76.17,18,0,8.05,AU,1619072281
502,Khorramabad,33.4878,48.3558,77.00,33,0,2.30,IR,1619072331


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create the Hotel DataFrame
hotel_df = optimal_df.loc[:, [ 'City', 'Country', 'Lat', 'Lng']]

# Add an empty hotel column to the DataFrame
hotel_df[ 'Hotel Name'] = ""

# Display to check
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
232,Maceió,BR,-9.6658,-35.7353,
262,Yulara,AU,-25.2406,130.9889,
322,Nicoya,CR,10.1483,-85.4520,
370,Alice Springs,AU,-23.7000,133.8833,
502,Khorramabad,IR,33.4878,48.3558,


In [8]:
# Set the base url and search parameters
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {'type': 'hotel',
          'keyword': 'hotel',
          'radius': 5000,
          'key': g_key}

In [9]:
#lat = 51.0833
#lng = 1.1833

#params[ 'location'] = f'{lat},{lng}'

#response = requests.get(base_url, params= params).json()

#pprint(response)

In [10]:
#print( response['results'][0]['name'])

In [11]:
# Search for hotels near the cities of interst
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row[ 'Lat']
    lng = row[ 'Lng']
    city = row[ 'City']
    
    # Add keyword to params dict
    params[ 'location'] = f'{lat},{lng}'
      
    # Assemble url, make API request, and print request to screen
    response = requests.get(base_url, params= params).json()
    print( f'Retrieving Results for Index { index}: { city}...')
    
   
    # Save hotel to DataFrame
    try:
        print( f'Closest hotel in { city} is',response['results'][0][ 'name'])
        hotel_df.loc[ index, "Hotel Name"] = response['results'][0][ 'name']

    # If no hotel is found, print message that city is being skipped
    except (KeyError, IndexError):
        print( 'No hotel found... skipping.')
        
    print( '-----------------------')
    
    
# Print End of Search
print( '-------End of Search-------')

Retrieving Results for Index 232: Maceió...
Closest hotel in Maceió is Meridiano Hotel
-----------------------
Retrieving Results for Index 262: Yulara...
Closest hotel in Yulara is Sails in the Desert
-----------------------
Retrieving Results for Index 322: Nicoya...
Closest hotel in Nicoya is Hotel El Regalo
-----------------------
Retrieving Results for Index 370: Alice Springs...
Closest hotel in Alice Springs is DoubleTree by Hilton Hotel Alice Springs
-----------------------
Retrieving Results for Index 502: Khorramabad...
Closest hotel in Khorramabad is Gahar motel
-----------------------
Retrieving Results for Index 515: Seoul...
Closest hotel in Seoul is Lotte Hotel Seoul
-----------------------
-------End of Search-------


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))